In [1]:
import numpy as np

In [2]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pickle as pkl
import torch
import random
import gc
from datetime import datetime
import psutil


'''PREFLIGHT SETUP'''
from functools import partial
print_flush = partial(print, flush=True)
torch.manual_seed(1)
random.seed(1)
np.random.seed(1)
'''PREFLIGHT SETUP'''


class Dataset_per_file(Dataset):
    def __init__(self, data_path, split, subset=0):
        super().__init__()
        self.data_path = data_path
        self.split = split
        self.dataset_file_paths,self.labels= self.build_dataset(subset=subset)
        self.length = len(self.dataset_file_paths)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        file_path = self.dataset_file_paths[idx]
        x = np.load(file_path)
        # print(np.isnan(x).any(), np.isnan(x[:, 0]).any(), np.isnan(x[:, 1]).any(), '---------------------------------------')
        ECG, PPG = torch.from_numpy(x[:, 0][None]).float(), torch.from_numpy(x[:, 1][None]).float()
        target = self.labels[idx]
        return ECG, PPG, target

    def build_dataset(self, subset=0):
        print_flush(f'\tpkl loading paths splits {self.split}')
        AF_paths = pkl.load(open(f'{self.data_path}/AF_v2/split_{self.split}.pkl', 'rb'))
        NSR_paths = pkl.load(open(f'{self.data_path}/NSR_v2/split_{self.split}.pkl', 'rb'))
        PVC_paths = pkl.load(open(f'{self.data_path}/PVC_v2/split_{self.split}.pkl', 'rb'))
        print_flush('\tpkl loading paths splits finished')

        if subset > 0:
            AF_paths = random.sample(AF_paths, k=subset)
            NSR_paths = random.sample(NSR_paths, k=subset)
            PVC_paths = random.sample(PVC_paths, k=subset)
        # print(len(AF_paths), len(NSR_paths), len(PVC_paths))
        
        AF_paths += PVC_paths
        AF_labels = np.ones((len(AF_paths)))
        NSR_labels = np.zeros((len(NSR_paths)))

        print(f'dataset built AF counts {len(AF_labels)}, NSR counts {len(NSR_labels)}', flush=True)

        all_paths = AF_paths + NSR_paths
        all_labels = torch.from_numpy(np.concatenate((AF_labels, NSR_labels))).long()
        return all_paths, all_labels



class Dataset_whole(Dataset):
    def __init__(self, data_path, split, subset):
        super().__init__()
        self.data_path = data_path
        self.split = split
        self.build_dataset()
        self.length = len(self.all_labels)
        self.subset = subset

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        
        if idx < self.AF_ECG.shape[0]:
            return self.AF_ECG[idx][None, :], self.AF_PPG[idx][None, :], self.all_labels[idx]
        elif idx >= self.AF_ECG.shape[0] and idx < (self.AF_ECG.shape[0] + self.PVC_ECG.shape[0]):
            offset = self.AF_ECG.shape[0]
            return self.PVC_ECG[idx-offset][None, :], self.PVC_PPG[idx-offset][None, :], self.all_labels[idx]
        else:
            offset = self.AF_ECG.shape[0] + self.PVC_ECG.shape[0]
            return self.PVC_ECG[idx-offset][None, :], self.PVC_PPG[idx-offset][None, :], self.all_labels[idx]

    def build_dataset(self):
        tstart = datetime.now()
        print_flush(f'\tloading... {self.split}')
        
        self.AF_ECG = torch.from_numpy(np.load(f'{self.data_path}/AF_v2/{self.split}_ECG_resampled2400.npy')).float()
        self.AF_PPG = torch.from_numpy(np.load(f'{self.data_path}/AF_v2/{self.split}_PPG_resampled2400.npy')).float()
        print_flush(f'\tAF loaded {self.split}')
        self.PVC_ECG = torch.from_numpy(np.load(f'{self.data_path}/PVC_v2/{self.split}_ECG_resampled2400.npy')).float()
        self.PVC_PPG = torch.from_numpy(np.load(f'{self.data_path}/PVC_v2/{self.split}_PPG_resampled2400.npy')).float()
        print_flush(f'\tPVC loaded {self.split}')
        self.NSR_ECG = torch.from_numpy(np.load(f'{self.data_path}/NSR_v2/{self.split}_ECG_resampled2400.npy')).float()
        self.NSR_PPG = torch.from_numpy(np.load(f'{self.data_path}/NSR_v2/{self.split}_PPG_resampled2400.npy')).float()
        print_flush(f'\tNSR loaded {self.split}')

        assert self.AF_ECG.shape == self.AF_PPG.shape
        assert self.NSR_ECG.shape == self.NSR_PPG.shape
        assert self.PVC_ECG.shape == self.PVC_PPG.shape

        print_flush(f'\tloading {self.split} finished t={datetime.now() - tstart}, mem used={psutil.virtual_memory()[3]/1000000000}')
        
        AF_labels = np.ones((self.AF_ECG.shape[0] + self.PVC_ECG.shape[0]))
        NSR_labels = np.zeros((self.NSR_ECG.shape[0]))
        self.all_labels = torch.from_numpy(np.concatenate((AF_labels, NSR_labels), axis=0)).long()

        print(f'dataset built AF counts {len(AF_labels)}, NSR counts {len(NSR_labels)}, total counts {len(self.all_labels)}', flush=True)



class Dataset_PPG(Dataset):
    def __init__(self, data_path, split, subset):
        super().__init__()
        self.data_path = data_path
        self.split = split
        self.build_dataset()
        self.length = len(self.all_labels)
        self.subset = subset

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        
        if idx < self.AF_PPG.shape[0]:
            return self.AF_PPG[idx][None, :], self.all_labels[idx]
        elif idx >= self.AF_PPG.shape[0] and idx < (self.AF_PPG.shape[0] + self.PVC_PPG.shape[0]):
            offset = self.AF_PPG.shape[0]
            return self.PVC_PPG[idx-offset][None, :], self.all_labels[idx]
        else:
            offset = self.AF_PPG.shape[0] + self.PVC_PPG.shape[0]
            return self.PVC_PPG[idx-offset][None, :], self.all_labels[idx]

    def build_dataset(self):
        tstart = datetime.now()
        print_flush(f'\tloading... {self.split}')
        
        self.AF_PPG = torch.from_numpy(np.load(f'{self.data_path}/AF_v2/{self.split}_PPG_resampled2400.npy')).float()
        print_flush(f'\tAF loaded {self.split}')
        self.PVC_PPG = torch.from_numpy(np.load(f'{self.data_path}/PVC_v2/{self.split}_PPG_resampled2400.npy')).float()
        print_flush(f'\tPVC loaded {self.split}')
        self.NSR_PPG = torch.from_numpy(np.load(f'{self.data_path}/NSR_v2/{self.split}_PPG_resampled2400.npy')).float()
        print_flush(f'\tNSR loaded {self.split}')

        print_flush(f'\tloading {self.split} finished t={datetime.now() - tstart}, mem used={psutil.virtual_memory()[3]/1000000000}')
        
        AF_labels = np.ones((self.AF_PPG.shape[0] + self.PVC_PPG.shape[0]))
        NSR_labels = np.zeros((self.NSR_PPG.shape[0]))
        self.all_labels = torch.from_numpy(np.concatenate((AF_labels, NSR_labels), axis=0)).long()

        print(f'dataset built AF counts {len(AF_labels)}, NSR counts {len(NSR_labels)}, total counts {len(self.all_labels)}', flush=True)



class Dataset_ECG(Dataset):
    def __init__(self, data_path, split, subset):
        super().__init__()
        self.data_path = data_path
        self.split = split
        self.build_dataset()
        self.length = len(self.all_labels)
        self.subset = subset

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        
        if idx < self.AF_ECG.shape[0]:
            return self.AF_ECG[idx][None, :], self.all_labels[idx]
        elif idx >= self.AF_ECG.shape[0] and idx < (self.AF_ECG.shape[0] + self.PVC_ECG.shape[0]):
            offset = self.AF_ECG.shape[0]
            return self.PVC_ECG[idx-offset][None, :], self.all_labels[idx]
        else:
            offset = self.AF_ECG.shape[0] + self.PVC_ECG.shape[0]
            return self.PVC_ECG[idx-offset][None, :], self.all_labels[idx]

    def build_dataset(self):
        tstart = datetime.now()
        print_flush(f'\tloading... {self.split}')
        
        self.AF_ECG = torch.from_numpy(np.load(f'{self.data_path}/AF_v2/{self.split}_ECG_resampled2400.npy')).float()
        print_flush(f'\tAF loaded {self.split}')
        self.PVC_ECG = torch.from_numpy(np.load(f'{self.data_path}/PVC_v2/{self.split}_ECG_resampled2400.npy')).float()
        print_flush(f'\tPVC loaded {self.split}')
        self.NSR_ECG = torch.from_numpy(np.load(f'{self.data_path}/NSR_v2/{self.split}_ECG_resampled2400.npy')).float()
        print_flush(f'\tNSR loaded {self.split}')

        print_flush(f'\tloading {self.split} finished t={datetime.now() - tstart}, mem used={psutil.virtual_memory()[3]/1000000000}')
        
        AF_labels = np.ones((self.AF_ECG.shape[0] + self.PVC_ECG.shape[0]))
        NSR_labels = np.zeros((self.NSR_ECG.shape[0]))
        self.all_labels = torch.from_numpy(np.concatenate((AF_labels, NSR_labels), axis=0)).long()

        print(f'dataset built AF counts {len(AF_labels)}, NSR counts {len(NSR_labels)}, total counts {len(self.all_labels)}', flush=True)


class Dataset_ori():
    def __init__(self,data_path,label_path):
        # self.root = root
        self.data_path = data_path
        self.label_path = label_path
        self.dataset,self.labelset= self.build_dataset()
        self.length = self.dataset.shape[0]
        # self.minmax_normalize()

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        step = self.dataset[idx,:]
        step = torch.unsqueeze(step, 0)
        # target = self.label[idx]
        target = self.labelset[idx]
        # target = torch.unsqueeze(target, 0)# only one class
        return step, target

    def build_dataset(self):
        '''get dataset of signal'''

        dataset = np.load(self.data_path)
        labelset = np.load(self.label_path)

        # dataset,labelset = shuffle(dataset,labelset)
        dataset = torch.from_numpy(dataset)
        labelset = torch.from_numpy(labelset)

        return dataset,labelset

In [4]:
data_path = '/labs/hulab/chengding_project_data/alarm_data_npy/'
train_dataset = Dataset_ori(data_path+'x_train2.npy',data_path+'y_train.npy')

In [6]:
train_dataset2 = Dataset_PPG('/labs/hulab/stark_stuff/ppg_ecg_project/data/', split='train', subset=0)

	loading... train
	AF loaded train
	PVC loaded train
	NSR loaded train
	loading train finished t=0:00:39.804008, mem used=160.600633344
dataset built AF counts 1518416, NSR counts 906523, total counts 2424939


In [9]:
val_dataset = Dataset_ori(data_path+'x_val2.npy',data_path+'y_val.npy')
val_dataset2 = Dataset_PPG('/labs/hulab/stark_stuff/ppg_ecg_project/data/', split='val', subset=0)

	loading... val
	AF loaded val
	PVC loaded val
	NSR loaded val
	loading val finished t=0:00:20.007798, mem used=172.725833728
dataset built AF counts 774369, NSR counts 444840, total counts 1219209


In [10]:
len(train_dataset2)+ len(val_dataset2)

3644148

In [11]:
len(val_dataset)+len(train_dataset)

8686029

In [13]:
import os
NSR_path = '/labs/hulab/chengding_project_data/AF_alarm/NSR_alarm_signals2/'
PVC_path = '/labs/hulab/chengding_project_data/AF_alarm/PVC_alarm_signals/'
AF_path = '/labs/hulab/chengding_project_data/AF_alarm/AF_alarm_signals2/'

NSR_files = os.listdir(NSR_path)
PVC_files = os.listdir(PVC_path)
AF_files = os.listdir(AF_path)

print(len(AF_files), len(PVC_files), len(NSR_files))

175216 2157599 1593981


In [14]:
len(AF_files)+ len(PVC_files)+ len(NSR_files)

3926796

In [15]:
!ls /labs/hulab/chengding_project_data/AF_alarm/

2012.csv		  af_alarm_stats.csv	 NSR_samples_new
2013.csv		  AF_samples		 PVC_alarms_final.pk
2014.csv		  AF_samples.pk		 PVC_alarm_signals
2015.csv		  bad_adibin		 PVC_alarms.pk
2016.csv		  bin2data.py		 PVC_sample_index2.csv
2017.csv		  check_bad_adbin2.py	 PVC_sample_index.csv
2018.csv		  NSR_alarms_all.pk	 PVC_samples
AFalarm_index.csv	  NSR_alarm_signals	 PVC_samples_final.pk
AFalarm_index_final2.csv  NSR_alarm_signals2	 PVC_samples_new
AFalarm_index_final3.csv  NSR_alarms_nonAF.pk	 shift_patients_bmi_old.pk
AFalarm_index_final.csv   NSR_sample_index2.csv  shift_patients_bmi.pk
AFalarm_index_v1.csv	  NSR_sample_index.csv	 table2.csv
AF_alarm_signals	  NSR_samples
AF_alarm_signals2	  NSR_samples_final.pk


In [16]:
NSR_path2 = '/labs/hulab/chengding_project_data/AF_alarm/NSR_alarm_signals/'
PVC_path2 = '/labs/hulab/chengding_project_data/AF_alarm/PVC_alarm_signals/'
AF_path2 = '/labs/hulab/chengding_project_data/AF_alarm/AF_alarm_signals/'

NSR_files2 = os.listdir(NSR_path2)
PVC_files2 = os.listdir(PVC_path2)
AF_files2 = os.listdir(AF_path2)

print(len(AF_files2), len(PVC_files2), len(NSR_files2))

3895134 2157599 1217240


In [17]:
len(AF_files2)+ len(PVC_files2)+ len(NSR_files2)

7269973

In [18]:
3895134 - 175216

3719918

In [20]:
7269973 / 3926796

1.8513752687942027

In [21]:
import os

In [25]:
!ls /labs/hulab/stark_stuff/ppg_ecg_project/data/AF_v3

DE191379874635896_20120815200026_2318_20120815200428.npy
DE191379874635896_20120815200026_2318_20120815200444.npy
DE191379874635896_20120815200026_2318_20120815200448.npy
DE191379874635896_20120815200026_2318_20120815200504.npy
DE191379874635896_20120815200026_2318_20120815200512.npy
DE191379874635896_20120815200026_2318_20120815200518.npy
DE191379874635896_20120815200026_2318_20120815200526.npy
DE191379874635896_20120815200026_2318_20120815200544.npy
DE191379874635896_20120815200026_2318_20120815200554.npy
DE191379874635896_20120815200026_2318_20120815200602.npy
DE191379874635896_20120815200026_2318_20120815200616.npy
DE191379874635896_20120815200026_2318_20120815200624.npy
DE191379874635896_20120815200026_2318_20120815200628.npy
DE191379874635896_20120815200026_2318_20120815200634.npy
DE191379874635896_20120815200026_2318_20120815200656.npy
DE191379874635896_20120815200026_2318_20120815200702.npy
DE191379874635896_20120815200026_2318_20120815200730.npy
DE191379874635896_2012081520002

DE191379874635896_20120816080836_2318_20120816184217.npy
DE191379874635896_20120816080836_2318_20120816184226.npy
DE191379874635896_20120816080836_2318_20120816184234.npy
DE191379874635896_20120816080836_2318_20120816184241.npy
DE191379874635896_20120816080836_2318_20120816184259.npy
DE191379874635896_20120816080836_2318_20120816184307.npy
DE191379874635896_20120816080836_2318_20120816184417.npy
DE191379874635896_20120816080836_2318_20120816184425.npy
DE191379874635896_20120816080836_2318_20120816184449.npy
DE191379874635896_20120816080836_2318_20120816184453.npy
DE191379874635896_20120816080836_2318_20120816184501.npy
DE191379874635896_20120816080836_2318_20120816184510.npy
DE191379874635896_20120816080836_2318_20120816184557.npy
DE191379874635896_20120816080836_2318_20120816184625.npy
DE191379874635896_20120816080836_2318_20120816184631.npy
DE191379874635896_20120816080836_2318_20120816184639.npy
DE191379874635896_20120816080836_2318_20120816184645.npy
DE191379874635896_2012081608083

DE191379874635896_20120818203748_2318_20120818233409.npy
DE191379874635896_20120818203748_2318_20120818233427.npy
DE191379874635896_20120818203748_2318_20120818233523.npy
DE191379874635896_20120818203748_2318_20120818233527.npy
DE191379874635896_20120818203748_2318_20120818233535.npy
DE191379874635896_20120818203748_2318_20120818233539.npy
DE191379874635896_20120818203748_2318_20120818233543.npy
DE191379874635896_20120818203748_2318_20120818233603.npy
DE191379874635896_20120818203748_2318_20120818233611.npy
DE191379874635896_20120818203748_2318_20120818233643.npy
DE191379874635896_20120818203748_2318_20120818233655.npy
DE191379874635896_20120818203748_2318_20120818233713.npy
DE191379874635896_20120818203748_2318_20120818233723.npy
DE191379874635896_20120818203748_2318_20120818233731.npy
DE191379874635896_20120818203748_2318_20120818233737.npy
DE191379874635896_20120818203748_2318_20120818233741.npy
DE191379874635896_20120818203748_2318_20120818233747.npy
DE191379874635896_2012081820374

DE609645727247408_20120829085055_2576_20120831144716.npy
DE609645727247408_20120829085055_2576_20120831154334.npy
DE609645727247408_20120829085055_2576_20120831154456.npy
DE609645727247408_20120829085055_2576_20120831161917.npy
DE609645727247408_20120829085055_2576_20120831162219.npy
DE609645727247408_20120829085055_2576_20120831162311.npy
DE609645727247408_20120829085055_2576_20120831162537.npy
DE609645727247408_20120829085055_2576_20120831162647.npy
DE609645727247408_20120829085055_2576_20120831162901.npy
DE609645727247408_20120829085055_2576_20120831165309.npy
DE609645727247408_20120829085055_2576_20120831174729.npy
DE609645727247408_20120829085055_2576_20120831174807.npy
DE609645727247408_20120829085055_2576_20120831174913.npy
DE609645727247408_20120829085055_2576_20120831175459.npy
DE609645727247408_20120829085055_2576_20120831175507.npy
DE609645727247408_20120829085055_2576_20120831175615.npy
DE609645727247408_20120829085055_2576_20120831180747.npy
DE609645727247408_2012082908505

DE609645727247408_20120919094653_2576_20120919131859.npy
DE609645727247408_20120919094653_2576_20120919131909.npy
DE609645727247408_20120919094653_2576_20120919131935.npy
DE609645727247408_20120919094653_2576_20120919131949.npy
DE609645727247408_20120919094653_2576_20120919131953.npy
DE609645727247408_20120919094653_2576_20120919132001.npy
DE609645727247408_20120919094653_2576_20120919132015.npy
DE609645727247408_20120919094653_2576_20120919132049.npy
DE609645727247408_20120919094653_2576_20120919132103.npy
DE609645727247408_20120919094653_2576_20120919132111.npy
DE609645727247408_20120919094653_2576_20120919132127.npy
DE609645727247408_20120919094653_2576_20120919132135.npy
DE609645727247408_20120919094653_2576_20120919132151.npy
DE609645727247408_20120919094653_2576_20120919132205.npy
DE609645727247408_20120919094653_2576_20120919132211.npy
DE609645727247408_20120919094653_2576_20120919132225.npy
DE609645727247408_20120919094653_2576_20120919132235.npy
DE609645727247408_2012091909465

In [3]:
x_train2 = np.load('/labs/hulab/chengding_project_data/alarm_data_npy/x_train2.npy')
y_train = np.load('/labs/hulab/chengding_project_data/alarm_data_npy/y_train.npy')
x_val2 = np.load('/labs/hulab/chengding_project_data/alarm_data_npy/x_val2.npy')
y_val = np.load('/labs/hulab/chengding_project_data/alarm_data_npy/y_val.npy')

In [4]:
print(x_train2.shape)

(8679807, 7200)


In [5]:
print(x_val2.shape)

(6222, 7200)


In [6]:
print(np.unique(y_train, return_counts=True))

(array([0., 1.], dtype=float16), array([4070350, 4609457]))


In [7]:
print(np.unique(y_val, return_counts=True))

(array([0, 1], dtype=int32), array([2599, 3623]))


In [8]:
8679807 + 6222

8686029

In [1]:
AF_path = '/labs/hulab/chengding_project_data/AF_alarm/AF_alarm_signals/'

In [2]:
import os

In [3]:
AF_files = os.listdir(AF_path)

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 30 15:13:36 2019

@author: sf109991
"""

import struct
import numpy as np
import pandas as pd

# param adibin_file: output of 'open' command in python3 (like _io.BufferedReader)
def parse_channels(adibin_file, dbg=False):
    FILE_HEADER_LENGTH = 68
    CHANNEL_HEADER_LENGTH = 96

    ADI_FILE_HEADER_FORMAT_STRING = "<4sldlllllddllll"
    ADI_CHANNEL_HEADER_FORMAT_STRING = "<32s32sdddd"

    # Start at the beginning of the buffer
    adibin_file.seek(0)
    
    # Read in overall file header
    file_header_bytes = adibin_file.read(FILE_HEADER_LENGTH)
    
    # Parse overall file header according to format string
    magic, version, secs_per_tick, year, month, day, hour, minute, second, \
     trigger, num_channels, samples_per_channel, time_czhannel, data_format \
    = struct.unpack(ADI_FILE_HEADER_FORMAT_STRING, file_header_bytes)
    
    if dbg == True:
        print("Magic:", magic.decode('utf-8'))
        print("Version:", version)
        print("secsPerTick:", secs_per_tick)
        print("Year:", year)
        print("Month:", month)
        print("Day:", day)
        print("Hour:", hour)
        print("Second:", second)
        print("Trigger:", trigger)
        print("NChannels:", num_channels)
        print("SamplesPerChannel:", samples_per_channel)
        print("DataFormat:", data_format)
        print('---')
    
    # Figure out length of rest of the file 
    if (data_format == 1): # 8 byte double
        bytes_per_sample = 8
        sample_format_string = 'd'
    elif (data_format == 2): # 4 byte float
        bytes_per_sample = 4
        sample_format_string = 'f'
    elif (data_format == 3): # 2 byte int
        bytes_per_sample = 2
        sample_format_string = 'h'
    else:
        return ValueError('DataFormat Not Coded to 1,2,or 3')
    
    # Order of entries in the array:
    data_names = ['ChannelIndex', 'ChannelTitle', 'Units', 'Scale', 
                            'Offset', 'RangeHigh', 'RangeLow', 'ChannelData']
    data_values_list = []

    # For every channel, read the header
    for channel_num in range(0, num_channels):
        channel_title_buffer = adibin_file.read(CHANNEL_HEADER_LENGTH)
        
        # Parse Channel Title Fields
        channel_title, units, scale, offset, range_high, range_low \
        = struct.unpack(ADI_CHANNEL_HEADER_FORMAT_STRING, channel_title_buffer)
        
        # Sanity Check
        if dbg == True:
            print("ChannelTitle:", channel_title.decode('utf-8'))
            print("Units:", units.decode('utf-8'))
            print("Scale:", scale)
            print("offset:", offset)
            print("RangeHigh:", range_high)
            print("RangeLow:", range_low)
            print('---')

        # Append channel title fields to list
        data_values_list.append([channel_num,
                                 channel_title.decode('utf-8').rstrip('\0'), 
                                 units.decode('utf-8').rstrip('\0'), 
                                 scale, 
                                 offset, 
                                 range_high, 
                                 range_low,
                                 []])

    # For every signal sample
    for sample in range(0, samples_per_channel):
        # For every channel
        for channel in range(0, num_channels):           
            data_values_list[channel][7].append(struct.unpack(sample_format_string,
                                                              adibin_file.read(bytes_per_sample))[0])
    
    # Return parsed list
    return list(map(lambda x: dict(zip(data_names, x)), data_values_list))


def test(filename):
    import matplotlib.pyplot as plt

    with open(filename, "rb") as adibin_file:
        ecg_channel_data = parse_channels(adibin_file, dbg=False)

    plt.plot(ecg_channel_data[0]['ChannelData'][0:300])
    plt.show()

def get_data_sample(path, channel_name):
    with open(path, 'rb') as adibin_file:
        data = parse_channels(adibin_file, dbg=False)
        channel = np.transpose(pd.DataFrame([data[j]['ChannelData'] for j in range(0,len(data))])) # read all 10 channels
        channel.columns = [i['ChannelTitle'] for i in data]
        return channel[channel_name].transpose().tolist()
#if __name__ == '__main__':
#    import sys
#    
#    try:
#        cmd = sys.argv[1]        
#    except:
#        InputError('Usage: python3 adibin2csv.py <filename.adibin/test filename.adibin>')
#        
#    if cmd == 'test':
#        try:
#            filename = sys.argv[2]
#        except:
#            InputError('Usage: python3 adibin2csv.py <filename.adibin/test filename.adibin>')
#
#        if filename.endswith('.adibin'):
#            test(filename)
#        else:
#            print('File extension must be ".adibin"')
#    elif cmd.endswith('.adibin'):
#        with open(cmd, 'rb') as adibin_file:
#            print(parse_channels(adibin_file))
#    else:
#        print('Usage: python3 adibin2csv.py <test/filename.adibin>')


In [5]:
file_path = AF_path
file = AF_files[0]

In [7]:
ECG = np.asarray(get_data_sample(file_path+file, 'I'))
PPG = np.asarray(get_data_sample(file_path+file, 'SPO2'))